# AutoML - Sentiment Analysis scenario: Train "the best" NLP Text Classification Multi-class model for a 'Sentiment Labeled Sentences' dataset.

**Requirements** - In order to benefit from this tutorial, you will need:
- A basic understanding of Machine Learning
- An Azure account with an active subscription. [Create an account for free](https://azure.microsoft.com/free/?WT.mc_id=A261C142F)
- An Azure ML workspace. [Check this notebook for creating a workspace](/sdk/resources/workspace/workspace.ipynb) 
- A Compute Cluster. [Check this notebook to create a compute cluster](/sdk/resources/compute/compute.ipynb)
- A python environment
- Installed Azure Machine Learning Python SDK v2 - [install instructions](/sdk/README.md#getting-started)

# 1. Connect to Azure Machine Learning Workspace

The [workspace](https://docs.microsoft.com/en-us/azure/machine-learning/concept-workspace) is the top-level resource for Azure Machine Learning, providing a centralized place to work with all the artifacts you create when you use Azure Machine Learning. In this section we will connect to the workspace in which the job will be run.

## 1.1. Import the required libraries

In [33]:
# Import required libraries
from azure.identity import DefaultAzureCredential
from azure.identity import InteractiveBrowserCredential
from azure.ml import MLClient

from azure.ml._constants import AssetTypes
from azure.ml.entities import JobInput

from azure.ml import automl
# from azure.ml.automl import text_classification

from pprint import pprint

## 1.2. Configure workspace details and get a handle to the workspace

To connect to a workspace, we need identifier parameters - a subscription, resource group and workspace name. We will use these details in the `MLClient` from `azure.ml` to get a handle to the required Azure Machine Learning workspace. We use the default [interactive authentication](https://docs.microsoft.com/en-us/python/api/azure-identity/azure.identity.interactivebrowsercredential?view=azure-python) for this tutorial. More advanced connection methods can be found [here](https://docs.microsoft.com/en-us/python/api/azure-identity/azure.identity?view=azure-python).

In [34]:
#Enter details of your AML workspace

# CDLTLL-GPU
# subscription_id = '381b38e9-9840-4719-a5a0-61d9585e1e91' #'<SUBSCRIPTION_ID>'
# resource_group = 'cesardl-automl-eastus2euap-resgrp' # '<RESOURCE_GROUP>'
# workspace = 'cesardl-dist-training-eastus-ws' # '<AML_WORKSPACE_NAME>'

# SAGAR
# subscription_id = "381b38e9-9840-4719-a5a0-61d9585e1e91" #'<SUBSCRIPTION_ID>'
# resource_group = "sasum_centraluseuap_rg" # '<RESOURCE_GROUP>'
# workspace = "sasum-centraluseuap-ws" # '<AML_WORKSPACE_NAME>'

# CDLTLL
# subscription_id = '102a16c3-37d3-48a8-9237-4c9b1e8e80e0' #'<SUBSCRIPTION_ID>'
# resource_group = 'automlpmdemo' # '<RESOURCE_GROUP>'
# workspace = 'cesardl-automl-centraluseuap-ws' # '<AML_WORKSPACE_NAME>'

# JUAMARTI
# subscription_id = "381b38e9-9840-4719-a5a0-61d9585e1e91"
# resource_group = "juamarti"
# workspace = "centraluseuap_phmantri"

subscription_id = "381b38e9-9840-4719-a5a0-61d9585e1e91"
resource_group = "sasum_centraluseuap_rg"
workspace = "sasum-centraluseuap-ws"


In [35]:
#get a handle to the workspace
credential = InteractiveBrowserCredential() # DefaultAzureCredential()
#credential = DefaultAzureCredential()
ml_client = MLClient(credential, subscription_id, resource_group, workspace)

# 2. Data

**NOTE:** In this PRIVATE PREVIEW we're defining the MLTable in a separate folder and .YAML file.
In later versions, you'll be able to do it all in Python APIs.

In [36]:
import os
import urllib
from zipfile import ZipFile

# Destination folders
training_mltable_path = "./training-mltable-folder/"
validation_mltable_path = "./validation-mltable-folder/"

# Download dataset files and copy within each MLTable folder

training_download_url = "https://raw.githubusercontent.com/dotnet/spark/main/examples/Microsoft.Spark.CSharp.Examples/MachineLearning/Sentiment/Resources/yelptrain.csv"
training_data_file = training_mltable_path + "yelp_training_set.csv"
urllib.request.urlretrieve(training_download_url, filename=training_data_file)

valid_download_url = "https://raw.githubusercontent.com/dotnet/spark/main/examples/Microsoft.Spark.CSharp.Examples/MachineLearning/Sentiment/Resources/yelptest.csv"
valid_data_file = validation_mltable_path + "yelp_validation_set.csv"
urllib.request.urlretrieve(valid_download_url, filename=valid_data_file)

print("Dataset files downloaded...")

Dataset files downloaded...


In [37]:
# Training MLTable defined locally, with local data to be uploaded
my_training_data_input = JobInput(type=AssetTypes.MLTABLE, path=training_mltable_path)

# Validation MLTable defined locally, with local data to be uploaded
my_validation_data_input = JobInput(type=AssetTypes.MLTABLE, path=validation_mltable_path)

# WITH REMOTE PATH: If available already in the cloud/workspace-blob-store
# my_training_data_input = JobInput(type=AssetTypes.MLTABLE, path="azureml://datastores/workspaceblobstore/paths/my_training_mltable")
# my_validation_data_input = JobInput(type=AssetTypes.MLTABLE, path="azureml://datastores/workspaceblobstore/paths/my_validation_mltable")    

# 3. Configure and run the AutoML NLP Text Classification Multiclass training job
In this section we will configure and run the AutoML job, for training the model.    

In [38]:
# Create the AutoML Image Classification Multiclass job with the related factory-function.

text_classification_job = automl.text_classification(
                            compute = "gpu-cluster",
                            # name="dpv2-nlp-text-classification-multiclass-job-01",
                            experiment_name = "dpv2-nlp-text-classification-experiment",
                            training_data = my_training_data_input,                           
                            validation_data = my_validation_data_input,  
                            target_column_name="Sentiment",
                            primary_metric = "accuracy",
                            tags={"owner": "cesardl"},
    
                            properties={
                                "_automl_internal_enable_mltable_quick_profile": True,
                                # "_automl_internal_scenario": "TextDNN-Candidate",
                            }
)

text_classification_job.set_limits(timeout=120, max_concurrent_trials=4)


compute is not a known attribute of class <class 'azure.ml._restclient.v2022_02_01_preview.models._models_py3.NlpVerticalDataSettings'> and will be ignored
experiment_name is not a known attribute of class <class 'azure.ml._restclient.v2022_02_01_preview.models._models_py3.NlpVerticalDataSettings'> and will be ignored
tags is not a known attribute of class <class 'azure.ml._restclient.v2022_02_01_preview.models._models_py3.NlpVerticalDataSettings'> and will be ignored
properties is not a known attribute of class <class 'azure.ml._restclient.v2022_02_01_preview.models._models_py3.NlpVerticalDataSettings'> and will be ignored


## 2.2 Run the CommandJob
Using the `MLClient` created earlier, we will now run this CommandJob in the workspace.

In [39]:
# Submit the AutoML job

returned_job = ml_client.jobs.create_or_update(text_classification_job)  # submit the job to the backend

print(f"Created job: {returned_job}")

Uploading training-mltable-folder (0.03 MBs): 100%|##########| 31262/31262 [00:00<00:00, 98000.95it/s]


Uploading validation-mltable-folder (0.03 MBs): 100%|##########| 30846/30846 [00:00<00:00, 77114.84it/s]


Exception: 'dict' object has no attribute 'data'.
Traceback (most recent call last):
  File "C:\Users\CESARDL.CESARDLSB2-MSFT\.conda\envs\automldpv2_wheel1\lib\site-packages\azure\ml\entities\_job\job.py", line 244, in _from_rest_object
    return AutoMLJob._load_from_rest(job_rest_object)
  File "C:\Users\CESARDL.CESARDLSB2-MSFT\.conda\envs\automldpv2_wheel1\lib\site-packages\azure\ml\entities\_job\automl\automl_job.py", line 78, in _load_from_rest
    return class_type._from_rest_object(obj)
  File "C:\Users\CESARDL.CESARDLSB2-MSFT\.conda\envs\automldpv2_wheel1\lib\site-packages\azure\ml\entities\_job\automl\nlp\text_classification_job.py", line 124, in _from_rest_object
    validation_data=JobInput(type=AssetTypes.MLTABLE, path=data_settings.validation_data.data.uri),
Attrib

JobParsingError: 'dict' object has no attribute 'data'

In [ ]:
# Get a URL for the status of the job
print("Open the following link to observe the AutoML training job/run:")

returned_job.services["Studio"].endpoint

# Next Steps
You can see further examples of other AutoML tasks such as Regression, Image-Object-Detection, NLP-Text-Classification, Time-Series-Forcasting, etc.